In [21]:
"""
IMPROVED HMM WITH BIGRAM MODELING
==================================
Incorporates the bigram transition approach from the high-performing solution.
"""

import numpy as np
import pickle
from collections import Counter
import re

In [22]:
import numpy as np
import pickle
from collections import Counter
import re

class ImprovedHangmanHMM:
    """
    Enhanced HMM that uses an interpolated trigram/bigram/unigram model,
    dynamically blended with a pattern-matching model for predictions.
    """
    
    def __init__(self, smoothing=1.0):
        self.smoothing = smoothing
        self.alphabet = 'abcdefghijklmnopqrstuvwxyz'
        self.letter_to_idx = {c: i for i, c in enumerate(self.alphabet)}
        self.idx_to_letter = {i: c for i, c in enumerate(self.alphabet)}
        
        # Probabilities for Unigram, Bigram, and Trigram models
        self.unigram_probs = np.ones(26) / 26
        self.bigram_probs = np.ones((26, 26)) / 26      # P(w_i | w_{i-1})
        self.trigram_probs = np.ones((26, 26, 26)) / 26 # P(w_i | w_{i-2}, w_{i-1})
        
        # Word storage for the pattern-matching sub-model
        self.words_by_length = {}
        self.trained = False
    
    def train(self, words):
        """Train the unigram, bigram, and trigram models on the corpus."""
        print("\nTraining HMM with Trigram, Bigram, and Unigram models...")
        
        # Initialize counts with smoothing to avoid zero probabilities
        unigram_counts = np.full(26, self.smoothing)
        bigram_counts = np.full((26, 26), self.smoothing)
        trigram_counts = np.full((26, 26, 26), self.smoothing)
        
        for word in words:
            word = word.lower()
            if not word or not all(c in self.alphabet for c in word):
                continue
            
            # Store word for pattern matching
            length = len(word)
            if length not in self.words_by_length:
                self.words_by_length[length] = []
            self.words_by_length[length].append(word)

            # Count unigrams
            for char in word:
                unigram_counts[self.letter_to_idx[char]] += 1.0
            
            # Count bigrams (w_{i-1} -> w_i)
            for i in range(len(word) - 1):
                prev_idx = self.letter_to_idx[word[i]]
                curr_idx = self.letter_to_idx[word[i+1]]
                bigram_counts[prev_idx, curr_idx] += 1.0
            
            # Count trigrams (w_{i-2}, w_{i-1} -> w_i)
            for i in range(len(word) - 2):
                p_prev_idx = self.letter_to_idx[word[i]]
                prev_idx = self.letter_to_idx[word[i+1]]
                curr_idx = self.letter_to_idx[word[i+2]]
                trigram_counts[p_prev_idx, prev_idx, curr_idx] += 1.0
        
        # Normalize all counts to probabilities, adding a small epsilon to avoid division by zero
        self.unigram_probs = unigram_counts / unigram_counts.sum()
        self.bigram_probs = bigram_counts / (bigram_counts.sum(axis=1, keepdims=True) + 1e-9)
        self.trigram_probs = trigram_counts / (trigram_counts.sum(axis=2, keepdims=True) + 1e-9)
        
        self.trained = True
        print(f"Trained on {len(words)} words.")
        print(f"Corpus contains words of lengths: {sorted(self.words_by_length.keys())}")

    def _get_pattern_probs(self, masked_word, guessed_letters):
        """Helper function to get probabilities from filtering the corpus."""
        length = len(masked_word)
        pattern_probs = np.zeros(26)
        num_matching_words = 0

        if length in self.words_by_length:
            pattern = masked_word.replace('_', '.')
            try:
                pattern_regex = re.compile(f'^{pattern}$')
            except re.error:
                return pattern_probs, 0 # Invalid regex pattern

            guessed_wrong = {l for l in guessed_letters if l not in masked_word}
            
            matching_words = [
                w for w in self.words_by_length[length] 
                if pattern_regex.match(w) and not any(c in guessed_wrong for c in w)
            ]
            num_matching_words = len(matching_words)

            if matching_words:
                # Count unguessed letters in the filtered word list
                letter_counts = Counter(c for w in matching_words for c in set(w) if c not in guessed_letters)
                if letter_counts:
                    total_counts = sum(letter_counts.values())
                    for letter, count in letter_counts.items():
                        pattern_probs[self.letter_to_idx[letter]] = count / total_counts
        
        return pattern_probs, num_matching_words

    def predict_letter_probabilities(self, masked_word, guessed_letters):
        """
        Predicts letter probabilities using a dynamic blend of pattern-matching 
        and an interpolated n-gram model.
        """
        if not self.trained:
            return np.ones(26) / 26
        
        # --- 1. Get probabilities from the INTERPOLATED N-GRAM MODEL ---
        # This model scores each potential letter based on its preceding context.
        lambda1, lambda2, lambda3 = 0.1, 0.3, 0.6  # Weights for unigram, bigram, trigram
        presence_probs = np.zeros(26)
        
        num_blanks = masked_word.count('_')
        if num_blanks == 0:
            return np.zeros(26)

        for i, char in enumerate(masked_word):
            if char == '_':
                # Get context: the two letters immediately preceding the blank
                prev_char = masked_word[i-1] if i > 0 and masked_word[i-1] != '_' else None
                p_prev_char = masked_word[i-2] if i > 1 and masked_word[i-2] != '_' else None
                
                prob_dist = self.unigram_probs # Default to unigram
                
                # If one preceding letter is known, interpolate bigram and unigram
                if prev_char:
                    prev_idx = self.letter_to_idx[prev_char]
                    bigram_dist = self.bigram_probs[prev_idx, :]
                    prob_dist = (1 - lambda1) * bigram_dist + lambda1 * self.unigram_probs
                
                # If two preceding letters are known, use the full trigram interpolation
                if p_prev_char and prev_char:
                    p_prev_idx = self.letter_to_idx[p_prev_char]
                    prev_idx = self.letter_to_idx[prev_char]
                    trigram_dist = self.trigram_probs[p_prev_idx, prev_idx, :]
                    bigram_dist = self.bigram_probs[prev_idx, :]
                    prob_dist = (lambda3 * trigram_dist) + (lambda2 * bigram_dist) + (lambda1 * self.unigram_probs)
                
                presence_probs += prob_dist

        # Normalize the summed probabilities from all blank positions
        if presence_probs.sum() > 0:
            presence_probs /= presence_probs.sum()

        # --- 2. Get probabilities from PATTERN MATCHING ---
        pattern_probs, num_matching_words = self._get_pattern_probs(masked_word, guessed_letters)

        # --- 3. DYNAMICALLY BLEND the two models ---
        if pattern_probs.sum() > 0:
            # If pattern matching is highly specific (few matching words), trust it more.
            pattern_weight = max(0.5, min(0.95, 0.95 - (num_matching_words / 500.0)))
            combined = pattern_weight * pattern_probs + (1.0 - pattern_weight) * presence_probs
        else:
            # If no words match the pattern, rely solely on the n-gram model
            combined = presence_probs
        
        # --- 4. Final Cleanup ---
        # Ensure already guessed letters have a probability of 0
        for letter in guessed_letters:
            if letter in self.letter_to_idx:
                combined[self.letter_to_idx[letter]] = 0
        
        # Final normalization to ensure probabilities sum to 1
        if combined.sum() > 0:
            combined /= combined.sum()
        
        return combined
    
    def save(self, filename='improved_hmm_model.pkl'):
        """Saves the trained HMM model to a file."""
        with open(filename, 'wb') as f:
            pickle.dump({
                'unigram_probs': self.unigram_probs,
                'bigram_probs': self.bigram_probs,
                'trigram_probs': self.trigram_probs,
                'words_by_length': self.words_by_length,
                'alphabet': self.alphabet,
                'letter_to_idx': self.letter_to_idx,
                'smoothing': self.smoothing
            }, f)
        print(f"Model saved to {filename}")
    
    def load(self, filename='improved_hmm_model.pkl'):
        """Loads a pre-trained HMM model from a file."""
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            self.unigram_probs = data['unigram_probs']
            self.bigram_probs = data['bigram_probs']
            self.trigram_probs = data['trigram_probs']
            self.words_by_length = data['words_by_length']
            self.alphabet = data['alphabet']
            self.letter_to_idx = data['letter_to_idx']
            self.smoothing = data['smoothing']
            self.trained = True
        print(f"Model loaded from {filename}")

In [23]:
if __name__ == "__main__":
    print("="*70)
    print("HANGMAN HMM TRAINER (Trigram Version)")
    print("="*70)
    
    # Load the corpus file
    try:
        with open('corpus.txt', 'r') as f:
            words = [line.strip().lower() for line in f if line.strip()]
        print(f"Loaded {len(words)} words from corpus.txt")
    except FileNotFoundError:
        print("Error: corpus.txt not found. Please ensure it's in the same directory.")
        exit()
    
    # Initialize and train the HMM
    hmm = ImprovedHangmanHMM(smoothing=1.0)
    hmm.train(words)
    
    # Save the trained model
    hmm.save('improved_hmm_model.pkl')
    
    # --- Test the model with some example cases ---
    print("\n" + "="*70)
    print("TESTING HMM PREDICTIONS")
    print("="*70)
    
    test_cases = [
        ("puzzl_", {'p','u','z','z','l'}),
        ("_ing", {'i','n','g'}),
        ("q__", {'q'}),
        ("appl_", {'a','p','p','l'}),
        ("pytho_", {'p','y','t','h','o'}),
    ]
    
    for masked, guessed in test_cases:
        probs = hmm.predict_letter_probabilities(masked, guessed)
        top_5 = sorted(enumerate(probs), key=lambda x: x[1], reverse=True)[:5]
        
        print(f"\nMasked: {masked}, Guessed: {guessed}")
        print("Top 5 letter predictions:")
        for idx, prob in top_5:
            if prob > 0:
                print(f"  {hmm.idx_to_letter[idx]}: {prob:.4f}")
    
    print("\n" + "="*70)
    print("HMM TRAINING AND TESTING COMPLETE!")
    print("="*70)

HANGMAN HMM TRAINER (Trigram Version)
Loaded 50000 words from corpus.txt

Training HMM with Trigram, Bigram, and Unigram models...
Trained on 50000 words.
Corpus contains words of lengths: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Model saved to improved_hmm_model.pkl

TESTING HMM PREDICTIONS

Masked: puzzl_, Guessed: {'l', 'p', 'u', 'z'}
Top 5 letter predictions:
  e: 0.3350
  i: 0.0965
  a: 0.0844
  y: 0.0820
  o: 0.0728

Masked: _ing, Guessed: {'g', 'i', 'n'}
Top 5 letter predictions:
  r: 0.2416
  t: 0.2414
  h: 0.2390
  j: 0.2373
  e: 0.0065

Masked: q__, Guessed: {'q'}
Top 5 letter predictions:
  u: 0.2127
  i: 0.1911
  a: 0.0971
  o: 0.0966
  c: 0.0958

Masked: appl_, Guessed: {'a', 'l', 'p'}
Top 5 letter predictions:
  e: 0.3482
  i: 0.2134
  o: 0.1524
  u: 0.0815
  y: 0.0731

Masked: pytho_, Guessed: {'y', 't', 'h', 'o', 'p'}
Top 5 letter predictions:
  m: 0.4814
  q: 0.4779
  r: 0.0081
  n: 0.0072
  l: 0.0056

HMM TRAINING AND TES

In [24]:
# ============================================================================
# PART 3: TESTING AND VALIDATION
# ============================================================================

def test_hmm(hmm, test_cases):
    """Test the HMM on some example cases."""
    print("\n" + "="*70)
    print("TESTING HMM PREDICTIONS")
    print("="*70)
    
    for masked_word, guessed in test_cases:
        probs = hmm.predict_letter_probabilities(masked_word, set(guessed))
        
        # Get top 5 predictions
        top_indices = np.argsort(probs)[-5:][::-1]
        top_letters = [(hmm.alphabet[i], probs[i]) for i in top_indices]
        
        print(f"\nMasked word: {masked_word}")
        print(f"Guessed: {guessed if guessed else 'none'}")
        print(f"Top 5 predictions:")
        for letter, prob in top_letters:
            print(f"  {letter}: {prob:.4f}")

In [25]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("="*70)
    print("HANGMAN HMM TRAINER")
    print("="*70)
    
    # Load corpus
    words = load_corpus('corpus.txt')
    
    # Analyze corpus
    letter_freq = analyze_corpus(words)
    
    # Initialize and train HMM
    hmm = HangmanHMM()
    hmm.train(words)
    
    # Save the model
    hmm.save('hmm_model.pkl')
    
    # Test cases
    test_cases = [
        ("_____", ""),           # Empty word, length 5
        ("_pp__", "p"),          # Word with known letter
        ("____", "eaios"),       # Many guessed letters
        ("a_____", "a"),         # First letter known
        ("_a____", "a"),         # Second letter known
        ("__e_____", "e"),       # Longer word
    ]
    
    test_hmm(hmm, test_cases)
    
    print("\n" + "="*70)
    print("HMM TRAINING COMPLETE!")
    print("="*70)
    print("\nModel saved as 'hmm_model.pkl'")
    print("You can now use this model in the RL agent.")

HANGMAN HMM TRAINER
Loaded 50000 words from corpus
Word length range: 1 to 24
Most common lengths: [(9, 6808), (10, 6465), (8, 6348), (11, 5452), (7, 5111), (12, 4292), (6, 3755), (13, 3094), (5, 2340), (14, 2019)]

Letter frequencies (top 10):
  e: 49224 (10.37%)
  a: 42110 (8.87%)
  i: 42068 (8.86%)
  o: 35829 (7.54%)
  r: 33619 (7.08%)
  n: 33314 (7.02%)
  t: 32191 (6.78%)
  s: 29044 (6.12%)
  l: 27406 (5.77%)
  c: 21718 (4.57%)

Training HMMs...
  Length 11: 5452 words
  Length 6: 3755 words
  Length 9: 6808 words
  Length 14: 2019 words
  Length 10: 6465 words
  Length 8: 6348 words
  Length 12: 4292 words
  Length 13: 3094 words
  Length 5: 2340 words
  Length 4: 1169 words
  Length 3: 388 words
  Length 7: 5111 words
  Length 15: 1226 words
  Length 2: 84 words
  Length 1: 46 words
  Length 20: 40 words
Trained models for 21 different word lengths
Model saved to hmm_model.pkl

TESTING HMM PREDICTIONS

Masked word: _____
Guessed: none
Top 5 predictions:
  a: 0.1069
  e: 0.0981
  